In [1]:
import boto3

from urllib.request import urlopen
from PIL import Image 

import pandas as pd
import json

In [ ]:
# df = pd.read_csv('../data/data_including_subscriber_and_thumbnail.csv')
# df = df[['video_id', 'views', 'subscriber_count', 'thumbnail']]
# df

In [ ]:
# df = df[df.views > 1000]
# df

In [2]:
df = pd.read_csv('../data/face_dectection_data.csv')

In [ ]:
# Initialize

# df['people_num'] = -1
# df['age'] = -1
# df['width'] = -1
# df['height'] = -1
# df['emotion'] = ''
# df['gender'] = ''

In [3]:
def detect_faces(url):

    client=boto3.client('rekognition')

    response = client.detect_faces(
        Image={'Bytes': urlopen(url).read()},
        Attributes=['ALL']
    )
    
    people_num = len(response['FaceDetails'])
    
    # 감지된 얼굴이 없는 경우
    if people_num == 0:
        return [int(people_num), -1, -1, -1, '', '']
    
    # 가장 큰 얼굴 하나만 데이터로 저장
    max_height = 0
    
    for faceDetail in response['FaceDetails']:
        # size
        height = faceDetail['BoundingBox']['Height']
        
        if height > max_height:
            max_height = height
        else: 
            continue
        
        width = faceDetail['BoundingBox']['Width']
        
        # age는 예상 나이 범위의 (low+high)/2
        age = (faceDetail['AgeRange']['Low'] + faceDetail['AgeRange']['High']) / 2
        
        # size
        width = faceDetail['BoundingBox']['Width']
        height = faceDetail['BoundingBox']['Height']
        
        # emotion
        emotion_confidence = 0
        emotion = ''
        
        for item in faceDetail['Emotions']:
            if item['Confidence'] > emotion_confidence:
                emotion_confidence = item['Confidence']
                emotion = item['Type']
        
        # gender
        gender = faceDetail['Gender']['Value']
    
    
    data = [int(people_num), int(age), width, height, emotion, gender]
    return data

In [4]:
def get_data(start_index, end_index):
    if end_index > len(df):
        end_index = len(df)
    for index in range(start_index, end_index):
        url = df['thumbnail'][index]
        data = detect_faces(url)
        df.loc[index, ['people_num', 'age', 'width', 'height', 'emotion', 'gender']] = data

In [5]:
get_data(0,10000)

df

HTTPError: HTTP Error 404: Not Found

In [6]:
df

,video_id,views,subscriber_count,thumbnail,people_num,age,width,height,emotion,gender
0,8e7GcsDEKEE,30038.0,98400,https://i.ytimg.com/vi/8e7GcsDEKEE/default.jpg,2,23,0.209781,0.265995,CALM,Male
1,FYoDak3VfF4,29102.0,98400,https://i.ytimg.com/vi/FYoDak3VfF4/default.jpg,1,9,0.209445,0.350828,SURPRISED,Male
2,FuTTOzwWo58,25022.0,98400,https://i.ytimg.com/vi/FuTTOzwWo58/default.jpg,1,18,0.210325,0.342480,CALM,Male
3,-48XAC_GDjk,49551.0,98400,https://i.ytimg.com/vi/-48XAC_GDjk/default.jpg,1,22,0.143989,0.230920,CALM,Male
4,mhehORE9WNA,74348.0,98400,https://i.ytimg.com/vi/mhehORE9WNA/default.jpg,2,5,0.136056,0.214099,CALM,Male
...,...,...,...,...,...,...,...,...,...,...
90769,K-sH6E9YeTw,137922.0,112000,https://i.ytimg.com/vi/K-sH6E9YeTw/default.jpg,-1,-1,-1.000000,-1.000000,NaN,NaN
90770,ucEs84amVm8,153005.0,112000,https://i.ytimg.com/vi/ucEs84amVm8/default.jpg,-1,-1,-1.000000,-1.000000,NaN,NaN
90771,jSK3H-dFfE0,151071.0,112000,https://i.ytimg.com/vi/jSK3H-dFfE0/default.jpg,-1,-1,-1.000000,-1.000000,NaN,NaN
90772,OEWk-sZG5j0,177515.0,112000,https://i.ytimg.com/vi/OEWk-sZG5j0/default.jpg,-1,-1,-1.000000,-1.000000,NaN,NaN


In [7]:
df.to_csv('../data/face_dectection_data.csv', index=False)